In [1]:

import json
import numpy as np
from datetime import datetime
import os
import time
from pytz import timezone
from pycromanager import Core
from pycromanager import Acquisition, multi_d_acquisition_events
import pandas as pd
import math
class scope_constant():
    piezo_focus_start_pos = -30
    piezo_focus_end_pos = 30
    piezo_step = 1.5

    scope_start_pos = -15
    scope_end_pos = 15
    scope_focus_end_pos = 30
    scope_focus_start_pos = -30
    scope_step = 1.5

    piezo_maxpro_start_pos = -15
    piezo_maxpro_end_pos = 15
    sharpen1 = np.array(([0, 1, 0],
                         [-1, 5, -1],
                         [0, -1, 0]), dtype="int")
    pos_per_slice = 4;
with open(os.path.join("config_file", "scope.json"), 'r') as r:
   scope_cfg=json.load(r)
system_path=os.getcwd()
slicePerSlide =1
focus_status = 0
alignment_status = 0
maketiles_status = 0
max_projection_status = 0
maxprojection_name =''
cancel_process = 0
live_plot=0
focus_bad=0
mock_align=0
ZDrive_safe_pos=scope_cfg[0]["ZDrive_safe_pos"]
XYStage_fluidics_safe_pos=scope_cfg[0]["XYStage_fluidics_safe_pos"]
piezo=scope_cfg[0]["piezo"]
XYStage_image_safe_pos=scope_cfg[0]["XYStage_image_safe_pos"]
scope_exposure_time_dict=scope_cfg[0]["scope_exposure_time_dict"]
stage_x_dir=scope_cfg[0]["stage_x_dir"]
stage_y_dir=scope_cfg[0]["stage_y_dir"]
pixelsize=scope_cfg[0]["pixel_size"]
imwidth=scope_cfg[0]["imwidth"]
overlap=scope_cfg[0]["overlap"]
maxprojection_drive=scope_cfg[0]["maxproject_drive"]
gene_target_channel =scope_cfg[0]["geneseq_focus_target_channel"]
hyb_target_channel=scope_cfg[0]["Hyb_focus_target_channel"]
align_channel=scope_cfg[0]["align_channel"]
pos_path = os.path.join(os.getcwd(),"scope_test")
cycle = "cycle00"
core=Core()


def get_pos_data(item):
    positions = item['DevicePositions']['array']
    pos_data = {}
    for pos in positions:
        pos_data.update(get_position_piezo(pos))
        pos_data.update({'position': item['Label']['scalar']})
   
    return pos_data


def get_position_piezo(pos):
    position_name = pos['Device']['scalar']
    position_value = pos['Position_um']['array']
    if position_name == 'ZDrive':
        return {'z': position_value[0]}
    elif position_name == 'XYStage':
        return {'x': position_value[0], 'y': position_value[1]}
    elif position_name == 'DA Z Stage':
        return {'piezo': position_value[0]}
    else:
        pass

### Test simple commands 

In [2]:
core.get_y_position()

-17012.5

In [3]:
core.get_position("DA Z Stage")

200

In [4]:
core.stop_stage_sequence('DA Z Stage')

In [5]:
with open(os.path.join( "cycle00.pos")) as f:
    d = json.load(f)
poslist = pd.DataFrame([get_pos_data(item) for item in d['map']['StagePositions']['array']])[
            ['position', 'x', 'y', 'z',
             'piezo']]
channel=["A","C","G","T"]
piezo_event_pos = np.arange(185, 215, 1.5)
piezo_event = []
for c in channel:
    for z in range(0, len(piezo_event_pos)):
        piezo_event.append({'axes': {'channel': c, 'z': z},
                            'config_group': ['Channel', c],
                            'exposure': scope_exposure_time_dict.get(c),
                            'z': piezo_event_pos[z]})

In [6]:
piezo_event

[{'axes': {'channel': 'A', 'z': 0},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 185.0},
 {'axes': {'channel': 'A', 'z': 1},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 186.5},
 {'axes': {'channel': 'A', 'z': 2},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 188.0},
 {'axes': {'channel': 'A', 'z': 3},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 189.5},
 {'axes': {'channel': 'A', 'z': 4},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 191.0},
 {'axes': {'channel': 'A', 'z': 5},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 192.5},
 {'axes': {'channel': 'A', 'z': 6},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 194.0},
 {'axes': {'channel': 'A', 'z': 7},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 195.5},
 {'axes': {'channel': 'A', 'z': 8},
  'config_group': ['Channel', 'A'],
  'exposure': 28,
  'z': 197.0},
 {'axes': {'channel': 'A', 'z': 9},
  'config_group': [

In [7]:
poslist

,position,x,y,z,piezo
0,Pos0,-16678.2,-17657.4,2620.0,200.0
1,Pos1,-16047.0,-17657.5,2620.0,200.0
2,Pos2,-16029.1,-17010.5,2620.0,200.0
3,Pos3,-16569.3,-17012.5,2620.0,200.0


## test acquisition code

In [10]:
image_path=os.path.join('geneseq01')
for index, row in poslist.iterrows():
    pos = row['position']
    x=row['x']
    y=row['y']
    core.set_xy_position(x, y)
    core.wait_for_device("XYStage")
    print( "XYStage ready")
    z=row['z']
    core.set_position("ZDrive", z)
    core.wait_for_device("ZDrive")
    print( "ZDrive ready")
    piezo_z=200
    core.set_position("DA Z Stage", piezo_z)
    core.wait_for_device("DA Z Stage")
    print("DA Z Stage")
    with Acquisition(directory=image_path, name=pos,show_display=False) as acq:
        acq.acquire(piezo_event)
        acq.mark_finished()    
    core.stop_stage_sequence('DA Z Stage')
    core.wait_for_device("DA Z Stage")
        

XYStage ready
ZDrive ready
DA Z Stage
XYStage ready
ZDrive ready
DA Z Stage
XYStage ready
ZDrive ready
DA Z Stage
XYStage ready
ZDrive ready
DA Z Stage
